In [15]:
import os
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

wine_path = ("/opt/datasets/"+"winequality-white.csv")
data = pd.read_csv(wine_path, delimiter=";")

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]



In [17]:
alpha = 0.5
l1_ratio = 0.5

In [18]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [19]:
with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8303146264282527
  MAE: 0.6307996574702415
  R2: 0.13291333254174142


## Saving Machine Learning Model : Serialization & Deserialization

In [ ]:
!pip install dill

In [21]:


import dill as pickle
filename = 'model_v2.pk'
path = '/opt/Web_apps/models/'

In [22]:
with open(path+filename, 'wb') as file:
    pickle.dump(lr,file)

### add pickle.dump(your_object, your_file, protocol=2)
dump in py 3 cannot load in py 2

In [23]:
with open(path+filename, 'wb') as file:
    pickle.dump(lr,file,protocol=2)

In [24]:
with open(path+filename, 'rb') as f:
    loaded_model = pickle.load(f)

In [25]:
with open(path+filename, 'rb') as f:
    loaded_model2 = pickle.load(f)

In [26]:
wine = test_x.head(1)
#iloc[0]
wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
4719,6.7,0.15,0.32,7.9,0.034,17.0,81.0,0.99512,3.29,0.31,10.0


In [27]:
type(wine)

pandas.core.frame.DataFrame

In [28]:
wine.values

array([[6.7000e+00, 1.5000e-01, 3.2000e-01, 7.9000e+00, 3.4000e-02,
        1.7000e+01, 8.1000e+01, 9.9512e-01, 3.2900e+00, 3.1000e-01,
        1.0000e+01]])

In [29]:
predicted_test = loaded_model.predict(wine.values)
predicted_test

array([5.9397091])

In [30]:
wine.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')